[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/lcl23-xnlm-lab/blob/main/notebooks/2_Attribution_Contrastive.ipynb)

In [ ]:
# Run in Colab to install local packages
%pip install ferret-xai inseq